In [1]:
'''Import libraries'''
import pandas as pd
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
import pickle

In [2]:
'''Load csv file'''
data = pd.read_csv("mbti_1.csv")


In [3]:
dataframe = data.copy()

In [4]:
def prepocessing_regex(text):
    text=re.sub(r'\|\|\|', r'', text) # removing |||
    text=re.sub(r'http\S+', r'', text) # removing http.
    text=text.replace(".", "").replace(",","").replace('+', '').replace(":","").replace('"','').replace("'",'')
    text=text.replace(";",'').replace('?',"").replace('_____','').replace('-','').replace('/','').replace("@","")
    text=text.replace("!",'')
    text=re.sub(r'[()]', '', text) # removing '(' or ')'
    text=re.sub(r'[<>]', '', text) # removing '<' or '>'
    text=text.replace("*","").lower()
    text=re.sub("[^a-zA-Z]"," ",text) #keep only words
    prepocessed_text=text
    
    stop_words = (stopwords.words('english')) 
    # mbti types will be removed in the posts for training and test 
    stop_words.extend(('infj', 'entp', 'intp', 'intj', 'entj', 'enfj', 'infp', 
                   'enfp', 'isfp', 'istp', 'isfj', 'istj', 'estp', 'esfp', 'estj', 'esfj',
                   'INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP','ISFP', 
                   'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'))
    stop_words=set(stop_words)
    word_tokens=word_tokenize(prepocessed_text)
    word_tokenized_preprocessed=[]

    for token in word_tokens:
        if token not in stop_words:
            word_tokenized_preprocessed.append(token)
    # converting list to string       
    s=word_tokenized_preprocessed
    converting_stopwords = ' '.join(map(str, s))
    stop_words=[converting_stopwords]
    
    return stop_words

In [5]:
def Lemmantization(prepocessed_words):
    n=WordNetLemmatizer()
    words_for_lemman=prepocessed_words
    lemmantized_word=[n.lemmatize(w) for w in words_for_lemman]
    # converting list to string
    to_str=lemmantized_word
    lemmantized_word=' '.join(map(str, to_str))
    
    return lemmantized_word

# Twitter prediction

In [6]:
import tweepy
import sys
import re
import numpy as np

# twitter API keys

In [7]:
def twitter_prediction(userID):
    consumer_key        = "ih9PqgrEHNlrZhR3gZS0oZKVz"
    consumer_secret     = "Do6vNjFBOQcXW6lmfC9XTL6E0r2JLZaaZ3cIMmjh0xn41a1mW0"
    access_token        = "1387268748738908166-CL5aXMb9i6SXPrYp2NFRM4lvNP2ylg"
    access_token_secret = "s6ChxUtAWGEEgdqcUOnN2qigX3AJ0xV2LCcOeOsMOmmsn"
    auth =tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api= tweepy.API(auth)

    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    all_tweets = []
    all_tweets.extend(tweets)
    oldest_id = tweets[-1].id
    while True:
        tweets = api.user_timeline(screen_name=userID, 
                               # 200 is the maximum allowed count
                               count=200,
                               include_rts = False,
                               max_id = oldest_id - 1,
                               # Necessary to keep full_text 
                               # otherwise only the first 140 words are extracted
                               tweet_mode = 'extended'
                               )
        if len(tweets) == 0:
            break
        oldest_id = tweets[-1].id
        all_tweets.extend(tweets)
        #print('N of tweets downloaded till now {}'.format(len(all_tweets)))
        
    jb=[]
    for info in all_tweets[:]:
        #print(info.full_text)
        jb.append(info.full_text)
        
        
    df=pd.DataFrame(jb,columns=['tweets'])
    df['processed']=df['tweets'].apply(prepocessing_regex)
    df['lemmantized']=df['processed'].apply(Lemmantization)
    df['one'] = df['lemmantized'].str.cat(sep=' ')
    vect=CountVectorizer(
                             analyzer = 'word',
                             tokenizer=None, 
                             preprocessor = None, 
                             stop_words = None, 
                             #ngram_range=(1, 3), 
                             #max_df=.8,
                             #min_df=2,                        
                             max_features = 2000
                            )
    
    x=[df['one'][0]]
    X =  vect.fit_transform(x) 
    
    #Logistic regression
    with open("IE_model_1.pkl","rb") as fr1a:
        IE_model_1 = pickle.load(fr1a)
    with open("NS_model_1.pkl","rb") as fr1b:
        NS_model_1 = pickle.load(fr1b)
    with open("FT_model_1.pkl","rb") as fr1c:
        FT_model_1 = pickle.load(fr1c)
    with open("JP_model_1.pkl","rb") as fr1d:
        JP_model_1 = pickle.load(fr1d)
        
       
        
        #SVM
    with open("IE_model_5.pkl","rb") as fr:
        IE_model_5 = pickle.load(fr)
    with open("NS_model_5.pkl","rb") as fr:
        NS_model_5 = pickle.load(fr)
    with open("FT_model_5.pkl","rb") as fr:
        FT_model_5 = pickle.load(fr)
    with open("JP_model_5.pkl","rb") as fr:
        JP_model_5 = pickle.load(fr)
  


    mbti=pd.DataFrame(index=[userID],columns=["LR","SVM"])#, columns=['User MBTI type'])
    mbti['LR']=(IE_model_1.predict(X))+(NS_model_1.predict(X))+(FT_model_1.predict(X))+(JP_model_1.predict(X))
    mbti['SVM']=(IE_model_5.predict(X))+(NS_model_5.predict(X))+(FT_model_5.predict(X))+(JP_model_5.predict(X))
    
    return mbti

In [8]:
twitter_prediction('BarackObama')  #"ENTP" 75 50  50

,LR,SVM
BarackObama,ENTJ,INFP


In [9]:
twitter_prediction('justinbieber')

,LR,SVM
justinbieber,INTJ,INFP


In [11]:
twitter_prediction('katyperry') 

,LR,SVM
katyperry,ENTP,INFP


In [12]:
twitter_prediction('rihanna') 

,LR,SVM
rihanna,INTJ,INFP
